In [1]:
#Bibliotecas a importar
import numpy as np
import math as math

### Sesión 4: Valuación de Opciones en tiempo discreto

In [69]:
#t_op: Tipo de opcion
# 1: Opción europea tipo call
# 2: Opción europea tipo put
# 3: Opción digital tipo call
# 4: Opción digital tipo put
t_op = 5
T = 1 #Tiempo de vigencia de la opción (en años)
r = 0.3 #tasa anual libre de riesgo
k = 4 #Precio strike. Funciona como el salto en el caso de las digitales
S_0 = 4 #Precio del subyacente a t=0
n = 3 #Particiones del árbol

u = 2 #Factor de aumento en el precio
d = 0.5 #Factor de disminución de precio

# Probabilidad en la medida libre de riesgo

In [70]:
delta_t = T / n
p = (math.exp(r * delta_t) - d) / (u - d) #Probabilidad de aumento en el precio
p

0.40344727871709846

# ESTO ES NOMAS PAL EJEMPLO X2

In [71]:
p=1/2

## Construcción del árbol

### Subyacente

Valor en el primer paso

In [28]:
#Matriz que almacena los valores del subyacente
arbol = np.zeros((n + 1, n + 1))
#El primer valor de la matriz es el subyacente a precio en t=0
arbol[0,0] = S_0
arbol #Visualziamos

array([[4., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

Valor del subyacente en el resto de los pasos

In [29]:
#For para llenar los valores de la matriz
for col in range(1, n +1):
    for ren in range(0, n +1):
        #Condicional para limitar matriz superior triangular
        if((col - ren) >= 0):
            arbol[ren, col] = S_0 *(( u ** (col - ren)) * (d ** (ren)))
arbol #Visualizamos

array([[ 4. ,  8. , 16. , 32. ],
       [ 0. ,  2. ,  4. ,  8. ],
       [ 0. ,  0. ,  1. ,  2. ],
       [ 0. ,  0. ,  0. ,  0.5]])

### Payoff

Primer paso

In [38]:
#Matriz que almacena el valor de los payoffs
payoffs = np.zeros((n+1, n+1))
#Sobreescribimos la última columna de la matriz de PAYOFSS
#IMPORTANTE: Agregar condicional para el caso del put (este es el caso particular del call)
if t_op==1:
    for ren in range(0, n+1):
        payoffs[ren, n] = max(arbol[ren, n] - k, 0)
elif t_op==2:
    for ren in range(0, n+1):
        payoffs[ren, n] = max(k-arbol[ren, n] , 0)
elif t_op==3:
    for ren in range(0, n+1):
        if (arbol[ren, n] > S_0):
            payoffs[ren, n] = k
        else:
            base[ren, n] = 0
elif t_op==4:
    for ren in range(0, n+1):
        if (arbol[ren, n] < S_0):
            payoffs[ren, n] = k
        else:
            payoffs[ren, n] = 0
elif t_op==5:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        suma=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            suma=suma+arbol[bajas,(i+1)]
        payoffs[ren,n]=max((suma/(n+1))-k,0)
        print(pos)
payoffs #Visualizamos

('u', 'u', 'u')
('u', 'u', 'd')
('u', 'd', 'u')
('u', 'd', 'd')
('d', 'u', 'u')
('d', 'u', 'd')
('d', 'd', 'u')
('d', 'd', 'd')


array([[ 0. ,  0. ,  0. , 11. ],
       [ 0. ,  0. ,  0. ,  5. ],
       [ 0. ,  0. ,  0. ,  2. ],
       [ 0. ,  0. ,  0. ,  0.5],
       [ 0. ,  0. ,  0. ,  0.5],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ]])

Valuamos backwards

In [56]:
#Sobreescribimos matriz de payoffs mediante el método iterativo
for i in range(1, n + 1):
    col = n - i
    for j in range(0, col +1): 
        payoffs[j, col] = math.exp(-r * delta_t) * (p * payoffs[2*j, col + 1] + (1 - p)*payoffs[2*j +1, col +1])
payoffs

array([[ 1.75944327,  3.78662973,  7.23869934, 11.        ],
       [ 0.        ,  0.10234134,  1.13104677,  5.        ],
       [ 0.        ,  0.        ,  0.22620935,  2.        ],
       [ 0.        ,  0.        ,  0.        ,  0.5       ],
       [ 0.        ,  0.        ,  0.        ,  0.5       ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

# ESTO ES NOMAS PAL EJEMPLO

In [57]:
#Sobreescribimos matriz de payoffs mediante el método iterativo
for i in range(1, n + 1):
    col = n - i
    for j in range(0, col +1): 
        payoffs[j, col] = 5/16 * (p * payoffs[2*j, col + 1] + (1 - p)*payoffs[2*j +1, col +1])
payoffs

array([[ 0.07247925,  0.45166016,  2.5       , 11.        ],
       [ 0.        ,  0.01220703,  0.390625  ,  5.        ],
       [ 0.        ,  0.        ,  0.078125  ,  2.        ],
       [ 0.        ,  0.        ,  0.        ,  0.5       ],
       [ 0.        ,  0.        ,  0.        ,  0.5       ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

## Valor de la opción

In [8]:
payoffs[0,0]

39.893012466730006

## Composición del portafolio

alfas

In [9]:
#Matriz que almacena los valores de las alfas
alfas = np.zeros((n, n ))
for ren in range(0,n):
    for col in range(0, n):
        #Condicional para limitar matriz superior triangular
        if((col - ren) >= 0):
            alfas[ren,col]=(payoffs[ren,col+1]-payoffs[ren+1,col+1])/(arbol[ren,col+1]-arbol[ren+1,col+1])
alfas #Visualizamos

array([[0.77885338, 1.        , 1.        ],
       [0.        , 0.58603981, 1.        ],
       [0.        , 0.        , 0.22511574]])

Betas

In [10]:
#Matriz que almacena los valores de las alfas
betas = np.zeros((n, n ))
for ren in range(0,n):
    for col in range(0, n):
        #Condicional para limitar matriz superior triangular
        if((col - ren) >= 0):
            betas[ren,col]=math.exp(-r * delta_t)*(payoffs[ren,col+1]-((payoffs[ren,col+1]-payoffs[ren+1,col+1])*arbol[ren,col+1])/(arbol[ren,col+1]-arbol[ren+1,col+1]))
betas #Visualizamos

array([[ -53.5693926 , -101.99585022, -108.30292136],
       [   0.        ,  -38.37183662, -108.30292136],
       [   0.        ,    0.        ,  -13.0256499 ]])

# Opciones asiáticas

In [3]:
pip- install itertools

ERROR: unknown command "-"
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install itertools

ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)
ERROR: No matching distribution found for itertools
Note: you may need to restart the kernel to use updated packages.


In [5]:
import itertools

In [14]:
a=list(itertools.product(['u','d'],repeat=3))
a

[('u', 'u', 'u'),
 ('u', 'u', 'd'),
 ('u', 'd', 'u'),
 ('u', 'd', 'd'),
 ('d', 'u', 'u'),
 ('d', 'u', 'd'),
 ('d', 'd', 'u'),
 ('d', 'd', 'd')]

In [22]:
list(a[0])

['u', 'u', 'u']

In [13]:
a[0][0]

'u'

In [16]:
np.zeros((2,2**2))

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [23]:
for i in range(0,2):
    print(i)

0
1


In [34]:
a=[[0,['u','d']]]

In [35]:
a

[[0, ['u', 'd']]]